Anlalyze times and hits of articles to upload 
a article in a timely manner to find a mentor

In [1]:
from bs4 import BeautifulSoup
import urllib.request as req
import pandas as pd
import time

In [60]:
url = 'https://www.clien.net/service/group/community?&od=T31&po=0'
res = req.urlopen(url) #소스를 res로 넣는다
soup = BeautifulSoup(res, "html.parser")

In [3]:
recommend = soup.select("div.list_content > div > div.list_symph > span")
category = soup.select("div > div.list_title > a > span.shortname.fixed")
title = soup.select("div > div.list_title > a > span.subject_fixed")
reply = soup.select("div.list_content > div > div.list_title > a")
writer = soup.select(" div.list_content > div > div.list_author")
viewCnt = soup.select("div.list_content > div > div.list_hit > span ")
dateTime =soup.select("div.list_content > div > div.list_time > span")
docNo = soup.select("div.list_content > div > div.list_title > a.list_subject")

In [66]:
# 라이터가 이미지인 경우에
# 리플이 안달린 경우에
reply = soup.select("div.list_content > div > div.list_title  ")
reply[1].text # try except
# 에러  나면 0으로 추가하도록

#div_content > div.list_content > div:nth-child(5) > div.list_title > a.list_subject > span.subject_fixed

'\n\n모공\n신천지는 까도된다는 판결\n\n\n'

In [72]:
#div_content > div.list_content > div:nth-child(8) > div.list_title > a.list_reply.reply_symph > span

int(str(reply[13]).split('<span class="rSymph05">')[1].split("</span>")[0])

<div class="list_title" data-role="list-title" data-toggle-custom="dropdown">
<a class="list_subject" data-role="cut-string" href="/service/board/park/14610822?od=T31&amp;po=0&amp;category=&amp;groupCd=community">
<span class="shortname fixed" title="모공">모공</span>
<span class="subject_fixed" data-role="list-title-text" title="이 음식 가격에 대해 어떻게 생각하세요 ??">이 음식 가격에 대해 어떻게 생각하세요 ??</span>
</a>
<span class="icon_pic fa fa-picture-o"></span>
<a class="list_reply reply_symph" data-role="ele-after" href="/service/board/park/14610822?od=T31&amp;po=0&amp;category=&amp;groupCd=community#comment-point" title="댓글10개">
<span class="rSymph05">10</span>
<strong class="color_red" data-role="observeCmCount"></strong>
<strong class="color_red" data-role="recentCmCount"></strong>
</a>
</div>

In [195]:

writer = soup.select(" div.list_content > div > div.list_author")

str(writer[28]).split('img alt="')[1].split('"')[0]


len(writer[2].text.split("\n")[3])
#그림 없는경우
writer[2].text.split("\n")[3]
writer[28].text.split("\n")[3]

# 그림이 있는 경우는 len이 0 이경우에는 
str(writer[28]).split('img alt="')[1].split('"')[0]


'초코클랸'

In [93]:
info=pd.DataFrame()
accum=pd.DataFrame()

for i in range(1489,2000):
    url = "https://www.clien.net/service/group/community?&od=T31&po=" + str(i)
    print(url)
    res = req.urlopen(url) #소스를 res로 넣는다
    soup = BeautifulSoup(res, "html.parser")

    recommend = soup.select("div.list_content > div > div.list_symph > span")
    category = soup.select("div > div.list_title > a > span.shortname.fixed")
    title = soup.select("div > div.list_title > a > span.subject_fixed")
    reply = soup.select("div.list_content > div > div.list_title")
    writer = soup.select(" div.list_content > div > div.list_author")
    viewCnt = soup.select("div.list_content > div > div.list_hit > span ")
    dateTime =soup.select("div.list_content > div > div.list_time > span")
    docNo = soup.select("div.list_content > div > div.list_title > a.list_subject")
    photoyn = soup.select("div.list_content > div > div.list_title")


    recList =[]
    catList =[]
    titList =[]
    repList =[]
    wriList =[]
    viewList =[]
    datList =[]
    docList = []
    phoList = []

    
    # 가끔 게시물이 30개가 아닌경우가 있음
    length=len(recommend)

    for ind in range(length):
        #추천수
        rec = recommend[ind].text
        #카테고리
        cat = category[ind].text
        #제목
        tit = title[ind].text      
        #조회수
        view = viewCnt[ind].text
        #날짜
        dat = dateTime[ind].text.split("\t")[13].rstrip('\n')
        #문서번호
        doc = docNo[ind]["href"].split("/")[4].split("?")[0]
        
        # if there's photo in article, also class "icon"
        # 아이콘 클래스가 있으면 사진이 있습니다
        if len(str(photoyn[0]).split("icon"))==2:
            pho = "Y"
        else:
            pho = "N"
        
        # if there's no reply, no data
        # 리플이 없으면 리플쪽 데이터는 없습니다.
        try:
            rep = int(str(reply[ind]).split('<span class="rSymph05">')[1].split('</span>')[0])
        except:
            rep = 0

        # if someone's nickname is image
        # 빤딱이가 붙으신 분들은 닉네임을 다른식으로 파싱합니다.
        
        if len(writer[ind].text.split("\n")[3]) == 0:
            wri = str(writer[ind]).split('img alt="')[1].split('"')[0]
        else:
            wri = writer[ind].text.split("\n")[3]
        
        # 파싱한 정보를 리스트에 넣습니다.
        recList.append(rec)
        catList.append(cat)
        titList.append(tit)
        repList.append(rep)
        wriList.append(wri)
        viewList.append(view)
        datList.append(dat)
        docList.append(doc)
        phoList.append(pho)

    # list into dataframe
    info = pd.DataFrame(columns=['추천수'],data = recList)
    info['게시판'] = catList
    info['제목'] = titList
    info['리플수'] = repList
    info['아이디'] = wriList
    info['조회수'] = viewList
    info['날짜'] = datList
    info['문서번호'] = docList
    info['사진유무'] = phoList
    if len(info) != 30:
        print("길이오류")
    
    accum = pd.concat([accum,info])
    info =pd.DataFrame()
    #한페이지 스크랩후에 1초를 쉬게합니다.
    time.sleep(1)

https://www.clien.net/service/group/community?&od=T31&po=1


In [94]:
accum

,추천수,게시판,제목,리플수,아이디,조회수,날짜,문서번호,사진유무
0,0,모공,"일본,유람선 승객 2 명 사망 신종 코로나 감염 확인 80 대 남녀,ㅠㅠ",3,king_cat,1074,2020-02-20 12:11:08,14611632,Y
1,6,모공,질병관리본부 임직원분들 힘내세요,1,kaka,253,2020-02-20 12:10:39,14611631,Y
2,2,모공,쿠팡이 결국 점유율을 가질 것이라.,16,Straight가자,1425,2020-02-20 12:10:26,14611629,Y
3,1,모공,대구지역 교회들당분간 예배 쉬면 좋겠는데 말이죠,5,피스버꾸,319,2020-02-20 12:10:26,14611628,Y
4,0,모공,"'끈적한 점액질이 호흡 막아', 예상 빗나간 코로나19 사망자 검시 결과",2,Mr.GO,1916,2020-02-20 12:09:58,14611627,Y
5,3,모공,펌) 메리 맬런 (일명 장티푸스 메리),2,의심과낭만,759,2020-02-20 12:09:23,14611626,Y
6,1,모공,"美연구팀, 코로나19 '단백질구조' 초정밀 지도 첫 성공..""획기적"".gisa",4,sano,1246,2020-02-20 12:09:16,14611625,Y
7,0,모공,신천지 때문에 코로나 전국적으로 퍼져나가게 생겼네요...ㄷㄷ,0,HoneST™,671,2020-02-20 12:09:07,14611624,Y
8,4,모공,오늘 문재인 대통령님하고 봉준호 감독님하고 오찬 한다네요,1,문라이징오,781,2020-02-20 12:09:02,14611623,Y
9,1,모공,"차클, 아베의 비선실세 일본희의",0,아름다워용,539,2020-02-20 12:08:25,14611622,Y


In [10]:


accum.reset_index(inplace = True, drop=True)
accum.to_csv('/clien.csv', index= False)